In [ ]:
%run D:\valkyrie_shared\python\lib\valkyrie\run_notebook_import.py

In [ ]:
#feature_mgr = FeatureMgr('20210101', '20210630', ['AGNCM', 'BAC PRN', 'WFC PRZ'] , 'eve_features3_2x0_quoter_cooldown_600')
feature_mgr = FeatureMgr('20210101', '20210630', stocks_good_dvd() , 'eve_features3_2x0_quoter_cooldown_600')
ret_ns = [int(1 * 2 * 60 * 6.5), int(3 * 2 * 60 * 6.5), int(5 * 2 * 60 * 6.5), int(10 * 2 * 60 * 6.5)]

ret_col, s2_col = 'risk_tv', 'cywa_mkt_s2h'
df_tv = feature_mgr.calc_rets('log', ret_col, ret_ns, s2_col)
df_tv['parent'] = df_tv['ticker'].apply(parent)

ret_col, s2_col = 'cywa_mkt_xh', 'cywa_mkt_s2h'
df_cy = feature_mgr.calc_rets('log', ret_col, ret_ns, s2_col)
df_cy['parent'] = df_cy['ticker'].apply(parent)
gc.collect()

In [ ]:

ins_nbs = []
ins_nbs.append(ml_data.InsNB('AGNCM',   ['BAC PRN', 'WFC PRZ']))
ins_nbs.append(ml_data.InsNB('BAC PRN', ['AGNCM', 'WFC PRZ']))
ins_nbs.append(ml_data.InsNB('WFC PRZ', ['AGNCM', 'BAC PRN']))
xcols = ['cywa_short_xh', 'cywa_long_xh']
ycol = 'cywa_mkt_xh_ret_780_n'
wcol = 'wgt_cywa_mkt_xh_ret_780_n'

In [ ]:
T = 4
i2df = feature_mgr.stk2df
mrf = ml_data.MultiRecField(i2df, ins_nbs, xcols, ycol, wcol, T)

In [ ]:
tn = mrf.n_samples_per_ins
i = 41116 + tn * 2
z = mrf[i][0]
### verify below manually
#z
#j = i % tn
#pd.concat([i2df[ticker][xcols + [ycol] + ['cywa_mkt_xh_ret_780_n']] for ticker in ['AGNCM','BAC PRN', 'WFC PRZ']], axis = 1).iloc[j:j+T]

In [ ]:
z.shape

# Test Speed

In [ ]:
len(mrf)

In [ ]:
with Timer('test1'):
    for i in range(len(mrf)):    
        z, _ = mrf[i]

In [ ]:
dm = ml_data.DataModule.from_dataset(mrf, mrf,128)
with Timer('data_loader'):
    data_loader = dm.get_dataloader(True)
    for i, z in enumerate(data_loader):
        y = z

# Test Speed II

In [ ]:
stks = stocks_good_dvd()
ins_nbs2 = []
for stk in stks:
    ins_nbs2.append(ml_data.InsNB(stk,   ['AGNCM']))

In [ ]:
mrf2 = ml_data.MultiRecField(i2df, ins_nbs2, xcols, ycol, wcol, T)

In [ ]:
len(mrf2)

In [ ]:
with Timer('test2'):
    for i in range(len(mrf2)):    
        z, _ = mrf2[i]

In [ ]:
dm = ml_data.DataModule.from_dataset(mrf2, mrf2, 128,num_workers = 2)
with Timer('data_loader'):
    data_loader = dm.get_dataloader(True)
    for i, z in enumerate(data_loader):
        y = z

In [ ]:
dm = ml_data.DataModule.from_dataset(mrf2, mrf2, 128, num_workers = 1)
with Timer('data_loader'):
    data_loader = dm.get_dataloader(True)
    for i, z in enumerate(data_loader):
        y = z

In [ ]:
dm = ml_data.DataModule.from_dataset(mrf2, mrf2, 128, num_workers = 8)
with Timer('data_loader'):
    data_loader = dm.get_dataloader(True)
    for i, z in enumerate(data_loader):
        y = z